# Structural manipulation and preparation tools

In [ ]:
from oganesson.ogstructure import OgStructure

og=OgStructure(file_name='examples/structures/MoS2.vasp')
og.add_atom_to_surface('Li').structure.to('MoS2_Li.vasp','poscar')

In [ ]:
from oganesson.ogstructure import OgStructure
og = OgStructure(file_name='examples/structures/Li3PO4_mp-13725.cif')
og.generate_neb('Li', r=3)


In [ ]:
from oganesson.genetic_algorithms import GA
from oganesson.ogstructure import OgStructure
from ase.build import bulk

Cu = bulk('Cu', 'fcc', a=3.6)
structure = OgStructure(Cu.repeat([4,4,4]))
structure.substitutions_random('Cu',{'Al':16, 'Cr':16, 'Ti':16, 'V':16})
structure()

# Machine learning

## Machine learning descriptors

In [ ]:
from oganesson.descriptors import BACD, SymmetryFunctions, DScribeSOAP, DScribeSineMatrix
from oganesson.ogstructure import OgStructure


bacd = BACD(OgStructure(file_name='examples/structures/mp-541001_LiInI4.cif'))
bacd.describe()

sf = DScribeSineMatrix(OgStructure(file_name='examples/structures/mp-541001_LiInI4.cif'))
sf.describe()


In [ ]:
from oganesson.descriptors import BACD, Describe
# from oganesson.descriptors import DScribeSOAP, DscribeACSF
from oganesson.ogstructure import OgStructure

bacd = BACD(OgStructure(file_name='examples/structures/Li3PO4_mp-13725.cif'))
print(bacd.describe())
# soap = DScribeSOAP(OgStructure(file_name='examples/structures/Li3PO4_mp-13725.cif'))
# acsf = DscribeACSF(OgStructure(file_name='examples/structures/Li3PO4_mp-13725.cif'))

print(Describe.describe(OgStructure(file_name='examples/structures/Li3PO4_mp-13725.cif'),BACD))

## Generation of machine learning datasets

In [ ]:
import requests
from oganesson.descriptors import Describe, BACD
from oganesson.ogstructure import OgStructure
from pymatgen.core.structure import Structure
from joltml import Experiment, Xgboost, Sklearn
import pandas as pd

headers = {
    'accept': 'application/json',
    'X-API-KEY': 'iGRUQOIQAcPMw00QWQKIEegfhF8O7Gmm'
}
materials_summary = requests.get('https://api.materialsproject.org/materials/summary/?formula=ABO3&deprecated=false&_per_page=1000&_skip=0&_limit=1000&_all_fields=true&is_stable=true', headers=headers)
materials_summary = materials_summary.json()['data']
datasets = {'material_ids':[],'structures':[],'bacd':[],'formation_energy_per_atom':[]}
for material in materials_summary:
    structure = OgStructure(Structure.from_dict(material['structure']))
    datasets['material_ids'] += [material['material_id']]
    datasets['structures'] += [structure]
    datasets['bacd'] += [Describe.describe(structure,descriptor=BACD)]
    datasets['formation_energy_per_atom'] += [material['formation_energy_per_atom']]


In [ ]:

import numpy as np
training_set = pd.DataFrame(datasets['bacd'])
experiment = Experiment(training_set.iloc[:800])
y = experiment.add_models([Xgboost()]).regression(targets=pd.DataFrame(datasets['formation_energy_per_atom'][:800]), splits=[0.8,0.2]).predict(training_set.iloc[:800])



To extract dynamics data from VASP OUTCAR files, there is an option to rely only on the available `OUTCAR` file, with the assumption that the first line of the `POSCAR` in the calculation had the correct assignment of species, and that there was not too many of them.

In [ ]:
from oganesson.io.vasp import Outcar

outcar = Outcar('examples/')
outcar.write_md_data()

However, it is better to obtain the species from the `POSCAR` file used in the calculation. So typically, one should provide both files to the `Outcar` class.


## Test sensitivity of descriptors

## Test translational and rotational symmetry of descriptors


In [ ]:
from oganesson.descriptors import BACD
from oganesson.ogstructure import OgStructure
bacd = BACD(OgStructure(file_name='examples/structures/Li3PO4_mp-13725.cif'))
print(bacd.is_invariant())

# Performing AIMD simulations

Here we apply a lithium-rich material, Li3PO4, to a temperature of 1000 K, and then calculate the diffusivities of the three elements.

In [ ]:
from oganesson.ogstructure import OgStructure
structure = OgStructure(file_name='examples/structures/LGPS_ChemMater_2018_30_4995_Opt.cif')
structure.simulate(temperature=1000,steps=1000,loginterval=1)
coeffs = structure.calculate_diffusivity()
print('Diffusion coefficients:', coeffs)


# Genetic algorithms

## Creating a population of materials, and performing evolutions

In [2]:
from oganesson.genetic_algorithms import GA
ga = GA(species=['Na']*4 + ['H']*4, rmax=5, population_size=5)
for i in range(10):
    ga.evolve()

og:Starting structural optimization using genetic algorithms..
og:Population size: 5
og:rmax threshold: 4.648075589890174


KeyboardInterrupt: 

## Evolving based on a given population

In [ ]:
from oganesson.genetic_algorithms import GA
from oganesson.ogstructure import OgStructure
from ase.build import bulk

Cu = bulk('Cu', 'fcc', a=3.6)
structure = OgStructure(Cu.repeat([2,2,2]))
print(structure())
ga = GA(population=structure.substitutions('Cu',{'Fe':4,'Cu':4}))
for i in range(10):
    ga.evolve()